In [0]:
## https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_data(a,d):  
  import requests
  def download_file_from_google_drive(id, destination):
      URL = "https://docs.google.com/uc?export=download"

      session = requests.Session()

      response = session.get(URL, params = { 'id' : id }, stream = True)
      token = get_confirm_token(response)

      if token:
          params = { 'id' : id, 'confirm' : token }
          response = session.get(URL, params = params, stream = True)

      save_response_content(response, destination)    

  def get_confirm_token(response):
      for key, value in response.cookies.items():
          if key.startswith('download_warning'):
              return value

      return None

  def save_response_content(response, destination):
      CHUNK_SIZE = 32768

      with open(destination, "wb") as f:
          for chunk in response.iter_content(CHUNK_SIZE):
              if chunk: # filter out keep-alive new chunks
                  f.write(chunk)
  download_file_from_google_drive(a,d)






In [0]:
def generate_dataset():
  from nltk.tokenize import sent_tokenize,word_tokenize
  from nltk.sentiment.vader import SentimentIntensityAnalyzer
  import nltk
  from nltk.data import load
  import numpy as np
  from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

  ##https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#TfidfTransformer-to-Compute-Inverse-Document-Frequency-IDF
  def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

  def extract_topn_from_vector(feature_names, sorted_items, topn):
    sorted_items = sorted_items[:topn]
    score_vals = [round(s, 3) for i,s in sorted_items]
    feature_vals = [feature_names[i] for i,s in sorted_items]

    results= {}
    for idx in range(len(feature_vals)):
      results[feature_vals[idx]]=score_vals[idx]

    return results

  users,rest,date,review,rating,label=[],[],[],[],[],[]
  with open('metadata', 'r', encoding="utf-8") as f:
    for l in f:
      a=l[:-1].split('\t')
      rating.append(float(a[2]))
      label.append(int(a[3]))

  with open('reviewContent', 'r', encoding="utf-8") as f:
    for l in f:
      a=l[:-1].split('\t')
      users.append(int(a[0]))
      rest.append(int(a[1]))
      date.append(str(a[2]))
      review.append(a[3]) 

  ## Features
  tagdict = load('help/tagsets/upenn_tagset.pickle')
  sid = SentimentIntensityAnalyzer()
  POS,length,avg_word_len,num_sent,avg_sent_len,positive_percent,neg_percent,bi,uni={},[],[],[],[],[],[],{},{}
  import pandas as pd
  data=pd.DataFrame()
  for i in POS:
    data[i]=POS[i]

  for i in tagdict.keys():
    POS['POS_'+i]=[]
  
  rmv_stop_words=[',','.','/','"','?',"'",'(',')','-','[',']','#','!','@','<','>','+',':',';']

  #BIGRAM
  vectorizer =TfidfVectorizer(stop_words=rmv_stop_words,ngram_range=(2,2))
  X = vectorizer.fit_transform(review)
  feature_names=vectorizer.get_feature_names()
  sorted_items=sort_coo(X.tocoo())
  keywords=extract_topn_from_vector(feature_names,sorted_items,100)          

  #unigram
  vectorizer_2 =TfidfVectorizer(stop_words=rmv_stop_words,ngram_range=(1,1))
  Y = vectorizer_2.fit_transform(review)
  feature_names_2=vectorizer_2.get_feature_names()
  sorted_items_2=sort_coo(Y.tocoo())
  keywords_2=extract_topn_from_vector(feature_names_2,sorted_items_2,100)

  for j in keywords:
    bi['bigram_'+j]=[0]*len(review)

  for j in keywords_2:
    uni['unigram_'+j]=[0]*len(review)

  dummy_index=0
  for i in review:
    length.append(len(i))
    
    ad=[len(z) for z in i.split(" ")]
    avg_word_len.append(sum(ad)/len(ad))
    
    a= i.split(" ")
    positive_percent.append(len([s for s in a if sid.polarity_scores(s)['compound']>0])/len(i))
    neg_percent.append(len([s for s in a if sid.polarity_scores(s)['compound']<0])/len(i))
    
    num_sent.append(len(sent_tokenize(i)))
    
    avg_sent_len.append(sum(len(word_tokenize(z)) for z in sent_tokenize(i))/len(sent_tokenize(i)))
    
    tags = nltk.pos_tag(nltk.word_tokenize(i))
    t=[]
    for j in tags:
      t.append(j[1])
    for j in tagdict.keys():
      if j in np.unique(np.array(t)):
        POS['POS_'+j].append(t.count(j)/len(t))
      else:
        POS['POS_'+j].append(0)

    for j in keywords:
      if j in i:
        bi['bigram_'+j][dummy_index]=1

    for j in keywords_2:
      if j in i:
        uni['unigram_'+j][dummy_index]=1
    dummy_index+=1

  #Dataset generation
  import pandas as pd
  data=pd.DataFrame()
  data['rating']=rating
  data['length of the review']=length
  data['average word length']=avg_word_len
  data['number of sentences']=num_sent
  data['average sentence length']=avg_sent_len
  for i in POS:
    data[i]=POS[i]
  data['positive %']=positive_percent
  data['negative %']=neg_percent
  for i in uni:
    data[i]=uni[i]
  for i in bi:
    data[i]=bi[i]
  data['label']=label

  data.to_csv('First data.csv',index=False)

  from google.colab import files as f
  f.download('First data.csv')



In [0]:
# The implementation of all the algorithms have been referenced from scikit-learn documentation.
def lr(X_train,y_train,X_test,y_test):
  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import accuracy_score
  LR = LogisticRegression(solver='lbfgs',max_iter=1000)
  LR.fit(X_train,y_train)

  
  print('\n#####################Logistic Regression#####################')
  from sklearn.metrics import classification_report, confusion_matrix
  print('==== Training:')
  y_pred=LR.predict(X_train)
  print(classification_report(y_train,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_train,y_pred))

  print('\n==== Testing:')
  y_pred=LR.predict(X_test)
  print(classification_report(y_test,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_test,y_pred))  

  from sklearn.metrics import accuracy_score
  return accuracy_score(y_test,y_pred)

In [0]:
def mnb(X_train,y_train,X_test,y_test):
  from sklearn.naive_bayes import MultinomialNB
  from sklearn.metrics import accuracy_score
  mnb=MultinomialNB()
  mnb.fit(X_train,y_train)
  # y_pred=mnb.predict(X_test)

  print('\n#####################Multinomial Naive Bayes#####################')
  from sklearn.metrics import classification_report, confusion_matrix
  print('==== Training:')
  y_pred=mnb.predict(X_train)
  print(classification_report(y_train,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_train,y_pred)) 

  print('\n==== Testing:')
  y_pred=mnb.predict(X_test)
  print(classification_report(y_test,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_test,y_pred))  

  from sklearn.metrics import accuracy_score
  return accuracy_score(y_test,y_pred)  

In [0]:
def linear_svm(X_train,y_train,X_test,y_test):
  from sklearn import svm
  clf = svm.SVC(gamma='auto',kernel='linear',cache_size=7000)
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  test_score=clf.score( X_test,y_test )
  
  print('\n#####################SVM#####################')
  from sklearn.metrics import classification_report, confusion_matrix
  print(classification_report(y_test,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_test,y_pred))    

  from sklearn.metrics import accuracy_score
  return accuracy_score(y_test,y_pred)


In [0]:
def rbf_svm(X_train,y_train,X_test,y_test):
  from sklearn import svm
  clf = svm.SVC(gamma='auto',kernel='rbf',cache_size=7000)
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  test_score=clf.score( X_test,y_test )
  
  print('\n#####################SVM#####################')
  from sklearn.metrics import classification_report, confusion_matrix
  print(classification_report(y_test,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_test,y_pred))    

  from sklearn.metrics import accuracy_score
  return accuracy_score(y_test,y_pred)

In [0]:
def dtree(X_train,y_train,X_test,y_test):
  from sklearn.tree import DecisionTreeClassifier
  dt=DecisionTreeClassifier()
  dt.fit(X_train,y_train)
  
  print('\n#####################Decision Tree#####################')
  from sklearn.metrics import classification_report, confusion_matrix
  print('==== Training:')
  y_pred=dt.predict(X_train)
  print(classification_report(y_train,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_train,y_pred)) 

  print('\n==== Testing:')
  y_pred=dt.predict(X_test)
  print(classification_report(y_test,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_test,y_pred))  

  from sklearn.metrics import accuracy_score
  return accuracy_score(y_test,y_pred)  

In [0]:
def rf(X_train,y_train,X_test,y_test):
  from sklearn.ensemble import RandomForestClassifier
  clf = RandomForestClassifier(n_estimators=30, max_depth=5,random_state=0)
  clf.fit(X_train,y_train)
  

  print('\n#####################Random Forest#####################')
  from sklearn.metrics import classification_report, confusion_matrix
  print('==== Training:')
  y_pred=clf.predict(X_train)
  print(classification_report(y_train,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_train,y_pred)) 

  print('\n==== Testing:')
  y_pred=clf.predict(X_test)
  print(classification_report(y_test,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_test,y_pred))  

  from sklearn.metrics import accuracy_score
  return accuracy_score(y_test,y_pred)  

In [0]:
#The implementation of below algorithm has been referenced from keras documentation
def nn(X_train,y_train,X_test,y_test):
  from tensorflow.python.util import deprecation
  deprecation._PRINT_DEPRECATION_WARNINGS = False
  import tensorflow as tf
  tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
  from keras import Sequential
  from keras.layers import Dense,Dropout
  num_hidden_layers=3
  classifier = Sequential()
  
  classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=X_train.shape[1]))
  for i in range(num_hidden_layers-1):
    classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dropout(0.2))

  classifier.add(Dense(1, activation='softmax', kernel_initializer='random_normal'))

  classifier.compile(optimizer ='adam',loss='binary_crossentropy',metrics =['accuracy'])
  classifier.fit(X_train,y_train, epochs=20,verbose=0)
  # y_pred=classifier.predict(X_test)
  # y_pred =(y_pred>0.5)

  print('\n#####################Neural Network#####################')
  from sklearn.metrics import classification_report, confusion_matrix
  print('==== Training:')
  y_pred=classifier.predict(X_train)
  print(classification_report(y_train,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_train,y_pred)) 

  print('\n==== Testing:')
  y_pred=classifier.predict(X_test)
  print(classification_report(y_test,y_pred))
  print('Confusion Matrix:\n\n',confusion_matrix(y_test,y_pred))  

  from sklearn.metrics import accuracy_score
  return accuracy_score(y_test,y_pred)

In [0]:
def pca(X1):
  acc=[]
  from copy import deepcopy as dc
  for i in range(1,X1.shape[1]+1):
    X=dc(X1)
    from sklearn.decomposition import PCA
    nf = i
    pca = PCA(n_components=nf)
    pca.fit(X)
    X = pca.fit_transform(X)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    acc.append(lr(X_train, y_train,X_test,  y_test))
  
  print("Max Acc:",max(acc) ,"for:",np.argmax(acc)+1)  

In [15]:
#Main function
def main():
  # generate_dataset()

  #download data
  a='1-pNvy5k7PTivT7qql-wN7PzEZvHa076c'
  d='data.csv'
  download_data(a,d)

  import pandas as pd
  import numpy as np

  data = pd.read_csv('data.csv')
  X1=data.iloc[:,:-1]
  y=data.iloc[:,-1]
  
  # pca(X1)

  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2)

  a1=mnb(X_train,y_train,X_test,y_test)

  # print("Before Scaling")
  # a1=mnb(X_train,y_train,X_test,y_test)
  # a2=dtree(X_train, y_train,X_test,y_test)
  # a3=lr(X_train, y_train,X_test,y_test)  
  # a4=rf(X_train,y_train,X_test,y_test)
  # a5=nn(X_train,y_train,X_test,y_test)

  # print("\nMinMax Scaler")
  # from sklearn.preprocessing import MinMaxtScaler
  # scaler = MinMaxScaler()
  # scaler.fit(X1)
  # X3 = scaler.transform(X1)
  # X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.2)
  # a2=dtree(X_train, y_train,X_test,y_test)
  # a3=lr(X_train, y_train,X_test,y_test)  
  # a4=rf(X_train,y_train,X_test,y_test)
  # a5=nn(X_train,y_train,X_test,y_test)

  # print("\nStandard Scaler")
  # from sklearn.preprocessing import StandardScaler
  # scaler = StandardScaler()
  # scaler.fit(X1)
  # X3 = scaler.transform(X1)
  # X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.2)
  # a2=dtree(X_train, y_train,X_test,y_test)
  # a3=lr(X_train, y_train,X_test,y_test)  
  # a4=rf(X_train,y_train,X_test,y_test)
  # a5=nn(X_train,y_train,X_test,y_test)

  # print("\nRobust Scaler")
  # from sklearn.preprocessing import RobustScaler
  # scaler = RobustScaler()
  # scaler.fit(X1)
  # X3 = scaler.transform(X1)
  # X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.2)
  # a2=dtree(X_train, y_train,X_test,y_test)
  # a3=lr(X_train, y_train,X_test,y_test)  
  # a4=rf(X_train,y_train,X_test,y_test)
  # a5=nn(X_train,y_train,X_test,y_test)

  from sklearn.preprocessing import Normalizer
  scaler=Normalizer()
  scaler.fit(X1)
  X2 = scaler.transform(X1)
  X_train, X_test, y_train, y_test = train_test_split(X2,y,test_size=0.2)
  a2=dtree(X_train, y_train,X_test,y_test)
  a3=lr(X_train, y_train,X_test,y_test)  
  a4=rf(X_train,y_train,X_test,y_test)
  a5=nn(X_train,y_train,X_test,y_test)

  print("\n\nTesting Accuracies:\nMultinomial NB:",a1,"\nDecision Tree:",a2,"\nLogistic Regression:",a3,"\nRandom Forest:",a4,"\nNeural Network:",a5)
 
if __name__ == "__main__":
    main()


#####################Multinomial Naive Bayes#####################
==== Training:
              precision    recall  f1-score   support

          -1       0.18      0.57      0.27     64260
           1       0.90      0.60      0.72    422618

    accuracy                           0.59    486878
   macro avg       0.54      0.58      0.49    486878
weighted avg       0.81      0.59      0.66    486878

Confusion Matrix:

 [[ 36522  27738]
 [170993 251625]]

==== Testing:
              precision    recall  f1-score   support

          -1       0.18      0.56      0.27     16206
           1       0.90      0.60      0.72    105514

    accuracy                           0.59    121720
   macro avg       0.54      0.58      0.49    121720
weighted avg       0.80      0.59      0.66    121720

Confusion Matrix:

 [[ 9138  7068]
 [42546 62968]]

#####################Decision Tree#####################
==== Training:
              precision    recall  f1-score   support

          -1    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

          -1       0.00      0.00      0.00     64434
           1       0.87      1.00      0.93    422444

    accuracy                           0.87    486878
   macro avg       0.43      0.50      0.46    486878
weighted avg       0.75      0.87      0.81    486878

Confusion Matrix:

 [[     0  64434]
 [     0 422444]]

==== Testing:


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

          -1       0.00      0.00      0.00     16032
           1       0.87      1.00      0.93    105688

    accuracy                           0.87    121720
   macro avg       0.43      0.50      0.46    121720
weighted avg       0.75      0.87      0.81    121720

Confusion Matrix:

 [[     0  16032]
 [     0 105688]]


Using TensorFlow backend.



#####################Neural Network#####################
==== Training:


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

          -1       0.00      0.00      0.00     64434
           1       0.87      1.00      0.93    422444

    accuracy                           0.87    486878
   macro avg       0.43      0.50      0.46    486878
weighted avg       0.75      0.87      0.81    486878

Confusion Matrix:

 [[     0  64434]
 [     0 422444]]

==== Testing:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00     16032
           1       0.87      1.00      0.93    105688

    accuracy                           0.87    121720
   macro avg       0.43      0.50      0.46    121720
weighted avg       0.75      0.87      0.81    121720

Confusion Matrix:

 [[     0  16032]
 [     0 105688]]


Testing Accuracies:
Multinomial NB: 0.5923923759447913 
Decision Tree: 0.7757065395990799 
Logistic Regression: 0.8681564245810056 
Random Forest: 0.8682878738087414 
Neural Network: 0.8682878738087414
